### Install libraries

In [3]:
from huggingface_hub import InferenceClient
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv
import os

### Load API key

In [4]:
# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_key = os.getenv("HUGGINGFACE_API_KEY")

### Load the model and tokenizer

In [5]:
# from transformers import LlamaForCausalLM, LlamaTokenizer

In [7]:
# model_name = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
# tokenizer = LlamaTokenizer.from_pretrained(model_name, use_auth_token=api_key)
# model = LlamaForCausalLM.from_pretrained(model_name, use_auth_token=api_key)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="JosephusCheung/LL7M")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("JosephusCheung/LL7M")
model = AutoModelForCausalLM.from_pretrained("JosephusCheung/LL7M")

### Set inference client

In [9]:
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")

In [11]:
import gradio as gr

def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    messages = [{"role": "system", "content": system_message}]
    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "system", "content": val[1]})
    
    messages.append({"role": "user", "content": message})

    response = " "

    for message in client.chat_completion(
        messages,
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = message.choices[0].delta.content
        response += token
        yield response

demo = gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(height=400),
    additional_inputs=[
        gr.Textbox(value="You are a friendly chatbot!", label="System message"),
    ],
)

if __name__ == "__main__":
    demo.launch()


/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packages/gradio/utils.py:1000: UserWarning: Expected 6 arguments for function <function respond at 0x16886f160>, received 3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packages/gradio/utils.py:1004: UserWarning: Expected at least 6 arguments for function <function respond at 0x16886f160>, received 3.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packages/gradio/helpers.py:947: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta/v1/chat/completions

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/huggingface/lib/python3.9/site-packa